In [250]:
from scipy.special import expit
from dotenv import load_dotenv
from openai import OpenAI
from typing import Any
from collections import Counter, defaultdict

import math
import re
import pandas as pd
import numpy as np
import tiktoken
import pickle
import faiss
import os

load_dotenv() 

EMBEDER_KEY = os.getenv('EMBEDER_KEY')
LLM_KEY = os.getenv('LLM_KEY')

TEXT_CHUNK_SIZE = 756
TAGS_ANNOTATIONS_CHUNK_SIZE = 256

TEXT_OVERLAP = 0.2
TAGS_ANNOTATIONS_OVERLAP = 0.75

K = 10 # Топ K при поиске в faiss
SK = 15 # Топ K документов для RAG

# 0. Подготовка

In [6]:
def data_to_storage(id_series: pd.Series, data_series: pd.Series):
    '''Функция парсит данные из датасета, собирая в JSON БД
    
    Args:
        id_series - столбец датасета с идентификатором документа
        data_series - стобец датасета с чанками
    Output:
        JSON БД
    '''
    storage = {}
    key_id = 0

    for row_num in range(len(data_series)):
        id_doc = id_series[row_num]
        data_document = data_series[row_num]

        for document in data_document:
            # Проверка на дополнительную вложенность
            if isinstance(document, list):
                for chunk in document:

                    storage[key_id] = (id_doc, chunk)
                    key_id += 1
            else:
                storage[key_id] = (id_doc, document)
                key_id += 1

    print(f"Storage ready, key from 0 to {key_id-1}")

    return storage

In [7]:
def get_embedding(text, dimensions=512):
    # Подключаемся к модели
    client = OpenAI(
        # Базовый url - сохранять без изменения
        base_url="https://ai-for-finance-hack.up.railway.app/",
        # Указываем наш ключ, полученный ранее
        api_key=EMBEDER_KEY,
    )
    # Формируем запрос к клиенту
    response = client.embeddings.create(
        # Выбираем любую допступную модель из предоставленного списка
        model="text-embedding-3-small",
        # Отправяем запрос
        input=text, 
        # Определяем размерность эмбединга
        dimensions = dimensions
    )
    # Формируем ответ на запрос и возвращаем его в результате работы функции
    return response.data[0].embedding


In [8]:
def get_batch_embeddings(texts, batch_size=32, dimensions=512):
    """Батчевые запросы к embedding API

    Args:
        texts: список строк
        batch_size: количество текстов в одном запросе
        dimensions: размерность эмбединга
    Returns:
        Список эмбеддингов (list[list[float]]).
    """
    client = OpenAI(
        base_url="https://ai-for-finance-hack.up.railway.app/",
        api_key=EMBEDER_KEY,
    )

    embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]

        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=batch,            
            dimensions=dimensions
        )

        # каждая запись в response.data соответствует одному элементу из batch
        batch_embeddings = [item.embedding for item in response.data]
        embeddings.extend(batch_embeddings)

    return embeddings

In [9]:
def normalize_vector(embedding: np.ndarray) -> np.ndarray:
    """
    Normalizes a given vector to have unit length.

    Args:
        embedding (np.ndarray): A NumPy array representing the vector to normalize.

    Returns:
        np.ndarray: A normalized vector with unit length.
    """

    norm = np.linalg.norm(embedding)
    if abs(norm) >= 1e-9: #защита от взрыва и погрешности
      embedding /= norm

    return embedding

Faiss

In [10]:
# Function to build an HNSW index
def build_faiss_hnsw_index(dimension, ef_construction=200, M=32):
    """
    build_faiss_hnsw_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Builds a FAISS HNSW index for cosine similarity.

    Parameters:
        dimension (int): Dimensionality of the embeddings.
        ef_construction (int): Trade-off parameter between index construction speed and accuracy.
        M (int): Number of neighbors in the graph.

    Returns:
        index (faiss.IndexHNSWFlat): Initialized FAISS HNSW index.
    """
    index = faiss.IndexHNSWFlat(dimension, M)  # HNSW index
    index.hnsw.efConstruction = ef_construction  # Construction accuracy
    index.metric_type = faiss.METRIC_INNER_PRODUCT  # Cosine similarity via normalized vectors
    return index

In [11]:
# Function to populate the FAISS index
def populate_faiss_index(index: faiss.Index, documents: dict, batch_size: int=20):
    """
    populate_faiss_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Populates the FAISS HNSW index with normalized embeddings from the dataset.

    Parameters:
        index (faiss.Index): FAISS index to populate.
        documents (pd.Series): documents like List[list[str]]
        batch_size (int): Number of questions to process at a time.
    """
    buffer = []
    i = 0

    for _, embedding in documents.items():
        embedding = normalize_vector(embedding)
        buffer.append(embedding)
        i += 1

        # Add embeddings to the index in batches
        if len(buffer) >= batch_size:
            index.add(np.array(buffer, dtype=np.float32))
            buffer = []

    # Add remaining embeddings
    if buffer:
        index.add(np.array(buffer, dtype=np.float32))

In [12]:
# Function to perform a search query
def search_faiss_index(embeddings_storage, query, k=5):
    """
    search_faiss_index: Add a description here.

    Args:
        # List the arguments with types and descriptions.

    Returns:
        # Specify what the function returns.
    """
    """
    Searches the FAISS index for the closest matches to a query.

    Parameters:
        embeddings_storage (faiss.Index): FAISS index to search.
        query (str): Query string to search.
        k (int): Number of closest matches to retrieve.

    Returns:
        indices (np.ndarray): Indices of the top-k results.
        distances (np.ndarray): Distances of the top-k results.
    """
    # Preprocess and normalize the query embedding
    query_embedding = get_embedding(query)
    query_embedding = np.array(query_embedding, dtype=np.float32)
    query_embedding = normalize_vector(query_embedding)
    # Search the embeddings_storage
    top_k_distances, top_k_indices = embeddings_storage.search(np.array([query_embedding], dtype=np.float32), k)

    # Match return format with that used in numpy storage search
    # Note that list manipulations will give an overhead
    top_k_indices_list = top_k_indices[0].tolist()
    top_k_distances_list = top_k_distances[0].tolist()

    return top_k_indices_list, top_k_distances_list

# 1. Подготовка Данных

In [13]:
# Тренировочный датасет
raw_data = pd.read_csv('data/train_data.csv')

raw_data.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"['Начать бизнес', 'Самозанятые', 'Свое дело', ...",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","['Защитить права', 'Банки', 'Банковская карта'...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"['Кредиты', 'Долги', 'Просрочки', 'Ипотека', '...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","['Инвестиции', 'Ценные бумаги', 'Фондовая бирж...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"['Займы', 'Долги', 'Риски', 'Защитить права']","## МФО больше нет в госреестре. Значит, она за..."


In [14]:
def second_preprocess(text: str) -> str:
    '''Удаляет лишние проблемы и переносы строк'''
    text = text.replace(' \n', '\n')
    text = text.replace('\n ', '\n')
    text = text.replace('\n', ' ')
    text = text.replace('  ', ' ').replace('   ', ' ')
    return text.strip()

def parse_question(block:str) -> tuple:
    '''Находит вопрос, извлекает его, удаляет из исходного текста
    
    Args:
        row - один блок до обработки
    Returns:
        tuple, где на 0 позиции вопрос (или '' если вопроса не было), на позиции 1 ответ
    
    '''
    candidats = block.split('\n')
    candidats = [row for row in candidats if row.strip()]

    if '?' in candidats[0]:
        question = candidats[0]
        answer = block.replace(question, '')
    else:
        answer = block
        question = ''

    question = second_preprocess(question)
    answer = second_preprocess(answer)
    
    return question, answer

def split_text(row:str):
    '''Функция сплитует по вопросу'''
    row = row.replace('###','')
    chunks = row.split('##')
    chunks = [chunk for chunk in chunks if chunk != '']
    return chunks

def parse_text(row:list):
    '''Получает на вход документ.
    Обрабатывает каждый блок документа его при помощи parse_question.

    Args:
        row - один документ
    Returns:
        Список с обработанными блоками, где каждый блок это tuple с вопросом и ответом. Вопрос может быть пустым.
    '''
    return [parse_question(bloc) for bloc in row]

def preprocess(df):
    # annotation
    df['annotation'] = df['annotation'].apply(lambda x: '' if x is np.nan else x)

    # tags
    df['tags'] = [row[1:-1].replace("'", "") for row in df['tags']]

    # text
    df['text'] = df['text'].str.replace(r'Обновлено \d{2}\.\d{2}\.\d{4} в \d{2}:\d{2}', '', regex=True)

    df['text'] = df['text'].apply(lambda x: split_text(x))

    df['text'] = df['text'].apply(lambda x: parse_text(x))

    return df

In [15]:
data = preprocess(raw_data)
data.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги","[(Кто такой самозанятый?, По закону самозаняты..."
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...","[(, Первым делом нужно попросить банк проверит..."
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...","[(, Не переоценивайте свои финансовые возможно..."
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...","[(, Просто прийти на биржу и купить ценные бум..."
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","[(МФО больше нет в госреестре. Значит, она зак..."


# 2. Чанкование

In [20]:
# Токенайзер для разбиения на чанки по длинне токенов
enc = tiktoken.get_encoding("cl100k_base")

In [21]:
def chunkinizer(
    question: str,
    answer: str, 
    chunk_size: int = 512,
    overlap_part: float = 0.2,
    enc: Any = enc
):
    """
    Разбивает текст на чанки по количеству токенов, используя tiktoken.

    Args:
        question: вопрос
        answer: ответ
        chunk_size: размер чанка в токенах.
        overlap_part: доля перекрытие в токенах между чанками.
        enc: токенизатор.

    Returns:
        Список чанков (строк).
    """
    # Количество токенов перекрытия
    overlap_tokens = int(chunk_size * overlap_part)
    # Токены вопроса и ответа
    tokens_answer = enc.encode(answer)
    tokens_question = enc.encode(question)
    # Количество токенов вопроса
    len_tokens_question = len(tokens_question)
    # Размера чанка, который заполняется ответом
    answer_chunk_size = chunk_size - len_tokens_question
        
    
    chunks = []

    start = 0

    while start < len(tokens_answer):
        # Поулчаем токены части ответа
        end = start + answer_chunk_size
        chunk_tokens = tokens_answer[start:end]

        # Текущий чанк Вопрос + Ответ
        concat_tokens = tokens_question + chunk_tokens
        chunk_text = enc.decode(concat_tokens)

        chunks.append(chunk_text)

        # Двигаем старт с учетом перекрытия
        start += answer_chunk_size - overlap_tokens

        if end >= len(tokens_answer):
            break

    return chunks


def vanila_chunkinizer(
    text: str,
    chunk_size: int = 512,
    overlap_part: float = 0.2,
    enc: Any = enc
):
    """
    Разбивает текст на чанки по количеству токенов, используя tiktoken.

    Args:
        text: исходный текст.
        chunk_size: размер чанка в токенах.
        overlap_part: доля перекрытие в токенах между чанками.
        enc: токенизатор.

    Returns:
        Список чанков (строк).
    """
    overlap_tokens = int(chunk_size * overlap_part)

    tokens = enc.encode(text)
    chunks = []

    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)

        start += chunk_size - overlap_tokens
        if end >= len(tokens):
            break

    return chunks

In [22]:
# Собираем чанки для тегов и аннотации
chunks_ta = [
    vanila_chunkinizer(
            f"{tags}. {annototions}", 
            chunk_size=TAGS_ANNOTATIONS_CHUNK_SIZE, 
            overlap_part=TAGS_ANNOTATIONS_OVERLAP
            ) 
    for annototions, tags in zip(data['annotation'], data['tags'])
    ]

data['annotation_tags_chunk'] = chunks_ta

In [25]:
# Собираем чанки для текста
chunks_t = list()

for row in data['text']:
    doc_chunks = list()
    for doc in row:
        question, answer = doc[0], doc[1]
        doc_chunks.append(chunkinizer(
            question=question, 
            answer=answer, 
            chunk_size=TEXT_CHUNK_SIZE, 
            overlap_part=TEXT_OVERLAP)
            )

    chunks_t.append(doc_chunks)

data['text_chunk'] = chunks_t

In [26]:
data.head()

,id,annotation,tags,text,annotation_tags_chunk,text_chunk
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги","[(Кто такой самозанятый?, По закону самозаняты...","[Начать бизнес, Самозанятые, Свое дело, Налоги...",[[Кто такой самозанятый?По закону самозанятый ...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...","[(, Первым делом нужно попросить банк проверит...","[Защитить права, Банки, Банковская карта, Риск...",[[Первым делом нужно попросить банк проверить ...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...","[(, Не переоценивайте свои финансовые возможно...","[Кредиты, Долги, Просрочки, Ипотека, Кредитная...",[[Не переоценивайте свои финансовые возможност...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...","[(, Просто прийти на биржу и купить ценные бум...","[Инвестиции, Ценные бумаги, Фондовая биржа, Ак...",[[Просто прийти на биржу и купить ценные бумаг...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","[(МФО больше нет в госреестре. Значит, она зак...","[Займы, Долги, Риски, Защитить права. Вы взяли...","[[МФО больше нет в госреестре. Значит, она зак..."


# 3. Подготовка json БД

In [27]:
# Создаем JSON  DB в формате [Сквозной идентификатор : (doc_id, chunk)] для тегов и ннотаций
storage_an_t = data_to_storage(
        id_series = data['id'],
        data_series = data['annotation_tags_chunk']
    )

Storage ready, key from 0 to 369


In [28]:
# Создаем JSON  DB в формате [Сквозной идентификатор : (doc_id, chunk)] для текстовых чанков
storage_t = data_to_storage(
        id_series = data['id'],
        data_series = data['text_chunk']
    )

Storage ready, key from 0 to 3048


In [29]:
# Пример собранных данныъ
print(storage_t[0], storage_t[1], sep='\n\n')

('doc_001', 'Кто такой самозанятый?По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль. Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины. Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.')

('doc_001', 'Сколько составляет налог на профессиональный доход?Есть два вида ставок для самозанятых. Какая именно будет использоваться в вашем случае, зависит от того, кто покупает ваши товары или услуги: * 4% — если деньги пришли от физического лица; * 6% — если оплата поступила от юридического лица или индивидуального предпринимателя. Эти ставки не будут меняться до конца 2028 года. Ученики Светланы — это в основном взрослые люди, которые хотят подтянуть разговорный английский перед отпуском или командировкой. За урок она берет 1000 рублей. Если Све

# 2. Получаем эмбединги

## 2.1 Для тегов + аннотация

In [30]:
# Соберем текст для батчевого запроса
texts = list()
for _, val in storage_an_t.items():
    texts.append(val[1])

len(texts)

370

In [31]:
# Запрос к openrouter
embeddings = get_batch_embeddings(texts)

In [33]:
with open('data/embeddings_ta_v1.pickle', 'wb') as f:
    pickle.dump(embeddings, f)

In [34]:
# Соберем временное хранилище векторов
embed_storage_an_t = dict()

for i in range(len(embeddings)):
    embed_storage_an_t[i] = np.array(embeddings[i], np.float32)

In [42]:
# Размерность полученных эмбедингов
len(embed_storage_an_t), embed_storage_an_t[0].shape[0]

(370, 512)

## 2.2 Для текста

In [38]:
# Соберем текст для батчевого запроса
texts = list()
for _, val in storage_t.items():
    texts.append(val[1])

len(texts)

3049

In [203]:
storage_t

{0: ('doc_001',
  'Кто такой самозанятый?По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль. Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины. Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.'),
 1: ('doc_001',
  'Сколько составляет налог на профессиональный доход?Есть два вида ставок для самозанятых. Какая именно будет использоваться в вашем случае, зависит от того, кто покупает ваши товары или услуги: * 4% — если деньги пришли от физического лица; * 6% — если оплата поступила от юридического лица или индивидуального предпринимателя. Эти ставки не будут меняться до конца 2028 года. Ученики Светланы — это в основном взрослые люди, которые хотят подтянуть разговорный английский перед отпуском или командировкой. За урок она берет 1000 рубл

In [148]:
# Запрос к openrouter
embeddings = get_batch_embeddings(texts, dimensions=1024)

In [152]:
# Соберем временное хранилище векторов
embed_storage_t = dict()

for i in range(len(embeddings)):
    embed_storage_t[i] = np.array(embeddings[i], np.float32)

with open('data/embed_storage_t_1024.pickle', 'wb') as f:
    pickle.dump(embed_storage_t, f)

# with open('data/embed_storage_t_1024.pickle', 'rb') as f:
#     embed_storage_t = pickle.load(f)

In [41]:
# Размерность полученных эмбедингов
len(embed_storage_t), embed_storage_t[0].shape[0]

(3049, 1024)

# 3. Инициализируем Faiss

## 3.1 Для тегов + аннотация

In [43]:
# Define the dimensions of the embedding vectors
embedding_dimension = 512  # Depends on the FastText model

# Build the HNSW index
hnsw_index_an_t = build_faiss_hnsw_index(embedding_dimension)

# Populate the index from pd.Series
populate_faiss_index(index=hnsw_index_an_t, documents=embed_storage_an_t)

In [44]:
data.loc[2]

id                                                                 doc_003
annotation               Самый надежный способ не оказаться в долгах — ...
tags                     Кредиты, Долги, Просрочки, Ипотека, Кредитная ...
text                     [(, Не переоценивайте свои финансовые возможно...
annotation_tags_chunk    [Кредиты, Долги, Просрочки, Ипотека, Кредитная...
text_chunk               [[Не переоценивайте свои финансовые возможност...
Name: 2, dtype: object

In [45]:
storage_an_t[0]

('doc_001',
 'Начать бизнес, Самозанятые, Свое дело, Налоги. Светлана из Казани дает частные уроки английского языка. Она неплохо зарабатывает, но весь ее доход — неофициальный. Из-за этого ей сложно получить кредит и визу в другую страну. Но недавно Светлана узнала, что можно получить статус самозанятого, платить небольшой налог — и проблем со справками о доходе не будет.Рассказываем, кто может зарегистрироваться как самозанятый и для чего это делать.')

In [46]:
data.loc[2, 'text']

[('',
  'Не переоценивайте свои финансовые возможности Прежде чем взять кредит, сделайте паузу и подумайте, насколько вам нужны эти деньги, можно ли обойтись без них и как вы будете возвращать полученную сумму. Учитывайте не только свою зарплату, пенсию или другие доходы, но и непредвиденные обстоятельства. Увольнение, болезнь или кризис могут резко ухудшить ваше финансовое положение. Важно заранее продумать, как в таких условиях справляться с выплатами по кредиту. Если вы берете крупный кредит на долгий срок, например ипотеку, подумайте о страховке. Она выручит вас в случае травмы, тяжелой болезни или потери работы.'),
 ('',
  'Не берите кредит в первом же банке Близость отделения или яркая реклама — не лучшие критерии для выбора банка. Изучите условия кредитов в нескольких организациях. Обратите внимание не только на процентную ставку, но и на другие параметры: срок кредитования, штрафы и пени за просрочку, необходимость страховки и ее стоимость. Уточните требования к заемщикам — нап

In [53]:
# Тестовый поиск по annotation
correct_id = 2 # рандомный идентификатор для примера annotation 
example = get_embedding(text=data.loc[correct_id, 'annotation']) # По тексту annotation получаем эмбединг
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 1) # В БД Tags+annotation ищем пример

assert correct_id == top_k_similarities.item(), 'Что то не работает =('

print(f"Требуемый документ из   БД:  {storage_an_t[correct_id][0]} ")
print(f"Загруженный документ из БД:  {storage_an_t[top_k_similarities.item()][0]} ")

Требуемый документ из   БД:  doc_003 
Загруженный документ из БД:  doc_003 


In [51]:
# Тестовый поиск по tags
correct_id = 2 # рандомный идентификатор для примера annotation 
example = get_embedding(text=data.loc[correct_id, 'tags']) # По тексту tags получаем эмбединг
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 10) # В БД Tags+annotation ищем пример

if not correct_id == top_k_similarities[0][0]:
    for i, el in enumerate(top_k_similarities[0]):   
        if el.item() == correct_id:
            print(f'Только на позиции {i}')
            break

print(f"Требуемый документ из БД:    {storage_an_t[correct_id][0]} ")
print(f"Загруженный документ из БД:  {storage_an_t[top_k_similarities[0][0]][0]} ")

Только на позиции 5
Требуемый документ из БД:    doc_003 
Загруженный документ из БД:  doc_061 


In [357]:
# Тестовый поиск по annotation
correct_id = 10 # рандомный идентификатор для примера annotation 
print(f'Тестовая анотация на основе которой составлен вопрос: {data.loc[correct_id, "annotation"]}')
query = 'Я попал в автомобильную аварию - что мне делать?'
example = get_embedding(query) 
example = normalize_vector(example)


top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 1) # В БД Tags+annotation ищем пример

assert correct_id == top_k_similarities.item(), 'Что то не работает =('

print(f"Требуемый документ из БД:    {storage_an_t[correct_id][0]} ")
print(f"Загруженный документ из БД:  {storage_an_t[top_k_similarities.item()][0]} ")

Тестовая анотация на основе которой составлен вопрос: Вы попали в небольшую аварию? Все живы-здоровы, а автомобили повреждены не очень сильно? Не нужно ждать сотрудников полиции, чтобы оформить происшествие. Вместе со вторым водителем вы можете сами задокументировать ДТП, чтобы пострадавший получил страховую выплату. Такая упрощенная процедура называется системой европротокола.
Требуемый документ из БД:    doc_011 
Загруженный документ из БД:  doc_011 


In [358]:
# Тестовый поиск по annotation
correct_id = 50 # рандомный идентификатор для примера annotation 
print(f'Тестовая анотация на основе которой составлен вопрос: {data.loc[correct_id, "annotation"]}')
query = 'Я внес платеж по кредиту вовремя, но банк мне прислали уведомдение о просрочке - почему?'
example = get_embedding(query) 
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_an_t.search(np.array([example], dtype=np.float32), 10) # В БД Tags+annotation ищем пример

if not correct_id == top_k_similarities[0][0]:
    for i, el in enumerate(top_k_similarities[0]):   
        if el.item() == correct_id:
            print(f'Только на позиции {i}')
            break

print(f"Требуемый документ из БД:    {storage_an_t[correct_id][0]} ")
print(f"Загруженный документ из БД:  {storage_an_t[top_k_similarities[0][0]][0]} ")

Тестовая анотация на основе которой составлен вопрос: Альбина гасит кредит через терминал для приема платежей. В этот раз она, как обычно, внесла деньги заранее, но банк прислал уведомление о просрочке. Оказалось, что банк, через который проходили платежи,лишился лицензиии деньги зависли. Разбираемся, почему так произошло и как действовать в подобной ситуации.
Только на позиции 6
Требуемый документ из БД:    doc_050 
Загруженный документ из БД:  doc_101 


## 3.2 Для текста

In [54]:
# Define the dimensions of the embedding vectors
embedding_dimension = 1024  # Depends on the FastText model

# Build the HNSW index
hnsw_index_t = build_faiss_hnsw_index(embedding_dimension)

# Populate the index from pd.Series
populate_faiss_index(index=hnsw_index_t, documents=embed_storage_t)

In [55]:
# Пример вопроса и ответа, по которому будем тестировать подход
data.loc[3, 'text'][2]

('В чем плюсы и минусы доверительного управления?',
 'Перед тем как обращаться к управляющему или покупать паи ПИФов, важно взвесить все за и против. **Плюсы** * Доверительное управление может принести больше дохода, чем банковский вклад. * Как и в случае с вкладом, вам не нужно постоянно следить за ситуацией на бирже, выбирать ценные бумаги или другие активы, определяться, когда их покупать, а когда продавать. Все решения за вас принимает доверительный управляющий или УК паевого фонда. **Минусы** * Прибыль не гарантирована. Инвестиции — это всегда риск, и чем больше потенциальная доходность, тем выше вероятность все потерять. Деньги в доверительном управлении не защищены государственной системой страхования вкладов. * Ваши доходы будут зависеть от решений посредника, выигрышных или неудачных. Поэтому выбирать доверительного управляющего или ПИФ стоит очень тщательно. Подробнее о рисках, с которыми сталкивается новичок на фондовом рынке, читайте в статье «Что нужно знать начинающему ин

In [56]:
# Тестовый поиск по ответу
correct_id = 3 # рандомный идентификатор 
user_query = data.loc[correct_id, 'text'][2][0] # [номер чанка][вопрос, ответ] - выбираем вопрос
print(f'User query: {user_query}')

example = get_embedding(text=user_query, dimensions=1024) 
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_t.search(np.array([example], dtype=np.float32), 10) # Ищем пример

if not correct_id == top_k_similarities[0][0]:
    for i, el in enumerate(top_k_similarities[0]):   
        if el.item() == correct_id:
            print(f'Только на позиции {i}')
            break
        
print(f"Требуемый документ из БД:    {data.loc[correct_id, 'id']} ")
print(f"Загруженный документ из БД:  {storage_t[top_k_similarities[0][0]][0]} ")

User query: В чем плюсы и минусы доверительного управления?
Требуемый документ из БД:    doc_004 
Загруженный документ из БД:  doc_004 


In [182]:
# Тестовый поиск
correct_id = 3 # рандомный идентификатор 
user_query = data.loc[correct_id, 'text'][2][1] # [номер чанка][вопрос, ответ] - выбираем ответ
example = get_embedding(text=user_query, dimensions=1024) 
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_t.search(np.array([example], dtype=np.float32), 10) # В БД Tags+annotation ищем пример

if not correct_id == top_k_similarities[0][0]:
    for i, el in enumerate(top_k_similarities[0]):   
        if el.item() == correct_id:
            print(f'Только на позиции {i}')
            break
        
print(f"Требуемый документ из БД:    {data.loc[correct_id, 'id']} ")
print(f"Загруженный документ из БД:  {storage_t[top_k_similarities[0][0]][0]} ")

Требуемый документ из БД:    doc_004 
Загруженный документ из БД:  doc_004 


In [72]:
# Тестовый поиск
correct_id = 3 # рандомный идентификатор 
user_query = "Кто такой самозанятый?" # [номер чанка][вопрос, ответ] - выбираем ответ
example = get_embedding(text=user_query, dimensions=1024) 
example = normalize_vector(example)

top_k_indices, top_k_similarities = hnsw_index_t.search(np.array([example], dtype=np.float32), 10) # В БД Tags+annotation ищем пример

# if not correct_id == top_k_similarities[0][0]:
#     for i, el in enumerate(top_k_similarities[0]):   
#         if el.item() == correct_id:
#             print(f'Только на позиции {i}')
#             break
        
# print(f"Требуемый документ из БД:    {data.loc[correct_id, 'id']} ")
# print(f"Загруженный документ из БД:  {storage_t[top_k_similarities[0][0]][0]} ")


In [81]:
top_k_indices[0]

array([-1.0097755, -1.2432148, -1.2744783, -1.3439044, -1.3783047,
       -1.3972595, -1.4023132, -1.4120059, -1.4144478, -1.414973 ],
      dtype=float32)

In [85]:
for sim, idx  in zip(top_k_indices[0], top_k_similarities[0]):
    print(f'Sim: {sim}', f"Chunk: {storage_t[idx][1]}")

Sim: -1.0097755193710327 Chunk: Кто такой самозанятый?По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль. Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины. Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.
Sim: -1.2432148456573486 Chunk: Кто может получить статус самозанятого?м нельзя заниматься никаким другим бизнесом. Кроме того, есть некоторые ограничения для людей, которые занимаются доставкой. Курьер может стать самозанятым только в том случае, если он развозит уже оплаченные товары и принимает плату только за услуги транспортировки. Или же у него есть кассовый аппарат от магазина или от компании-производителя, товар которых он доставляет. Это единственный вариант, когда самозанятый курьер имеет право взять деньги за саму покупку. Са

In [84]:
idx, sim

(np.float32(-1.0097755), np.int64(0))

# 4. Подключение к LLM, проработка прототипа (вопрос -> эмбединг -> ответ)

In [58]:
def answer_generation(question):
    # Подключаемся к модели
    client = OpenAI(
        # Базовый url - сохранять без изменения
        base_url="https://ai-for-finance-hack.up.railway.app/",
        # Указываем наш ключ, полученный ранее
        api_key=LLM_KEY,
    )

    system_prompt = """Ты RAG-ассистент, вежливый помошник по банковским, финансовым и прочим вопросам. 
    Пользователь задает тебе вопрос с указанием ответить на него (Ответь на вопрос:). 
    Так же в сообщении пользователя будет указана информация, которую ты должен использовать для ответа.
    Отвечай открыто и интересно, по возможности приводи примеры!
    Начинай с приветсвия пользователя!"""

    # Формируем запрос к клиенту
    response = client.chat.completions.create(
        # Выбираем любую допступную модель из предоставленного списка
        model="openrouter/google/gemma-3-27b-it",
        # Формируем сообщение
        messages=[
            
                {"role": "system", "content": system_prompt},
                {"role": "user", 
                "content": [
                    {
                        "type": "text",
                        "text": f"Ответь на вопрос: {question}"
                    }
                ]}
        ]
    )
    # Формируем ответ на запрос и возвращаем его в результате работы функции
    return response.choices[0].message.content

In [59]:
def z_logistic(s, k=1.0):
    z = (s - s.mean()) / (s.std(ddof=0) + 1e-9)
    return expit(k * z)

## 4.1 Поулчаем вопрос его эмбединги и осуществляем поиск в faiss

In [60]:
# Получаем тестовый вопрос
correct_id = 3 # рандомный идентификатор 
user_query = data.loc[correct_id, 'text'][2][0] # [номер чанка][вопрос, ответ] - выбираем вопрос
print(f'User query: {user_query}')

User query: В чем плюсы и минусы доверительного управления?


In [61]:
# Эмбединг вопроса для текста
example_1024 = get_embedding(text=user_query, dimensions=1024) 
example_1024 = normalize_vector(example_1024)

# Эмбединг вопроса для тега + аннотации
example_512 = get_embedding(text=user_query, dimensions=512) 
example_512 = normalize_vector(example_512)

In [62]:
# Топ K вопросов по тексту
top_k_similarities_text, top_k_indices_text = hnsw_index_t.search(np.array([example_1024], dtype=np.float32), K) # В БД Tags+annotation ищем пример

text_sim = dict()
for idx, sim in zip(top_k_indices_text[0], top_k_similarities_text[0]):
    text_sim[idx.item()] = sim.item()

# Ток K вопросам по аннотациям + теги 
top_k_similarities_key_annot, top_k_indices_key_annot = hnsw_index_an_t.search(np.array([example_512], dtype=np.float32), K) # В БД Tags+annotation ищем пример

tags_annot_sim = dict()
for idx, sim in zip(top_k_indices_key_annot[0], top_k_similarities_key_annot[0]):
    tags_annot_sim[idx.item()] = sim.item()

## 4.2 TF-IDF

In [267]:
# ---------------------------
# 1) Токенизация
# ---------------------------
def tokenize(text):
    text = text.lower()
    tokens = re.findall(r"[a-zа-я0-9]+", text)
    return tokens

# ---------------------------
# 2) Построение индекса
# docs — список строк (документов)
# ---------------------------
class BM25:
    def __init__(self, docs, k1=1.5, b=0.75):
        self.k1 = k1
        self.b = b
        self.docs = [tokenize(doc) for doc in docs]
        self.N = len(self.docs)

        # длина документа
        self.doc_lengths = [len(doc) for doc in self.docs]
        self.avgDL = sum(self.doc_lengths) / self.N

        # TF-грамматика: list[Counter]
        self.tf = [Counter(doc) for doc in self.docs]

        # DF: количество документов, где встречается слово
        df = defaultdict(int)
        for doc in self.docs:
            for word in set(doc):
                df[word] += 1

        # IDF
        self.idf = {}
        for word, freq in df.items():
            # Okapi IDF с защитой от отрицательности
            self.idf[word] = math.log(1 + (self.N - freq + 0.5) / (freq + 0.5))

    # ---------------------------
    # 3) Счёт BM25 для одного документа
    # ---------------------------
    def score(self, query, index):
        query_tokens = tokenize(query)
        score = 0.0
        doc_tf = self.tf[index]
        doc_len = self.doc_lengths[index]

        for term in query_tokens:
            if term not in doc_tf:
                continue
            tf = doc_tf[term]
            idf = self.idf.get(term, 0)

            denom = tf + self.k1 * (1 - self.b + self.b * doc_len / self.avgDL)
            score += idf * (tf * (self.k1 + 1)) / denom

        return score

    # ---------------------------
    # 4) Поиск top-k
    # ---------------------------
    def search(self, query, top_k=5):
        scores = [(i, self.score(query, i)) for i in range(self.N)]
        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        return scores[:top_k]
    
BM25_TOPK = 10 

def bm25_screach(query:str, bm25, BM25_TOPK:int = BM25_TOPK):

    bm25_results = bm25.search(query, top_k=BM25_TOPK)

    bm25_results = pd.DataFrame(
        bm25_results,
        columns=['top_k_indices_text', 'bm25_score']
        )
    
    return bm25_results


# Все наши документы (теги)
docs = list()
for _, val in storage_t.items():
    docs.append(val[1])

bm25 = BM25(docs)

bm25_screach(
    query = "В чем минусы и плюсы доверительного управления?",
    bm25=bm25
)

,top_k_indices_text,bm25_score
0,29,24.160303
1,1409,14.824194
2,27,14.747822
3,105,13.549853
4,2027,12.952948
5,927,12.674418
6,28,12.667782
7,2602,12.280617
8,26,12.104516
9,1909,11.811735


In [245]:
# Все наши документы (теги)
docs = list()
for _, val in storage_t.items():
    docs.append(val[1])

bm25 = BM25(docs)

query = "В чем минусы и плюсы доверительного управления?"
bm25_results = bm25.search(query, top_k=10)

for doc_id, score in bm25_results:
    print(score, docs[doc_id])

bm25_results = pd.DataFrame(
    bm25_results,
    columns=['top_k_indices_text', 'bm25_score']
    )

24.160302707080795 В чем плюсы и минусы доверительного управления?Перед тем как обращаться к управляющему или покупать паи ПИФов, важно взвесить все за и против. **Плюсы** * Доверительное управление может принести больше дохода, чем банковский вклад. * Как и в случае с вкладом, вам не нужно постоянно следить за ситуацией на бирже, выбирать ценные бумаги или другие активы, определяться, когда их покупать, а когда продавать. Все решения за вас принимает доверительный управляющий или УК паевого фонда. **Минусы** * Прибыль не гарантирована. Инвестиции — это всегда риск, и чем больше потенциальная доходность, тем выше вероятность все потерять. Деньги в доверительном управлении не защищены государственной системой страхования вкладов. * Ваши доходы будут зависеть от решений посредника, выигрышных или неудачных. Поэтому выбирать доверительного управляющего или ПИФ стоит очень тщательно. Подробнее о рисках, с которыми сталкивается новичок на фондовом рынке, читайте в статье «Что нужно знать на

## 4.2 Ранжирование чанков и формирования rag_message

__TEXT__

bm25

In [246]:
# Датасет в рамках которого будем крутить скоры
bm25_results['doc_id'] = bm25_results['top_k_indices_text'].apply(lambda idx: storage_t[idx][0])

# Номрализуем косинусную близость - чам больше - тем ближе, т.е. лучше
bm25_results['bm25_score'] = z_logistic(bm25_results[['bm25_score']]) * 0.2

In [247]:
bm25_results

,top_k_indices_text,bm25_score,doc_id
0,29,0.189349,doc_004
1,1409,0.109296,doc_163
2,27,0.108203,doc_004
3,105,0.090979,doc_012
4,2027,0.082531,doc_233
5,927,0.078668,doc_106
6,28,0.078577,doc_004
7,2602,0.073320,doc_301
8,26,0.070979,doc_004
9,1909,0.067163,doc_222


Embedings

In [89]:
# Датасет в рамках которого будем крутить скоры
df_text_result = pd.DataFrame({
    'top_k_indices_text' : top_k_indices_text[0].T,
    'cos_scores' : top_k_similarities_text[0].T,
    'doc_id' : [storage_t[idx.item()][0] for idx in top_k_indices_text[0]]
    })

# Номрализуем косинусную близость - чам больше - тем ближе, т.е. лучше
df_text_result['cos_scores_m'] = z_logistic(df_text_result[['cos_scores']]) 
# Взвешиваем скор
df_text_result['cos_scores_m'] = df_text_result['cos_scores_m'] * 0.65

# номер чанка (бж текст), косинусная близость, документ, нормированная косинусная близость
df_text_result

,top_k_indices_text,cos_scores,doc_id,cos_scores_m
0,30,-0.810738,doc_004,0.638537
1,29,-0.830464,doc_004,0.603303
2,28,-0.831735,doc_004,0.600864
3,31,-0.858537,doc_004,0.544920
4,33,-0.887900,doc_004,0.475245
5,34,-0.954206,doc_004,0.307025
6,27,-0.976039,doc_004,0.255817
7,32,-1.003471,doc_004,0.198557
8,26,-1.007861,doc_004,0.190235
9,2114,-1.013576,doc_245,0.179766


__Annotation, Tags__

In [88]:
df_tags_annot_result = pd.DataFrame({
    'top_k_indices_key_annot' : top_k_indices_key_annot[0].T,
    'cos_scores_ka' : top_k_similarities_key_annot[0].T,
    'doc_id' : [storage_an_t[idx.item()][0] for idx in top_k_indices_key_annot[0]]
    })

# Могут быть дубли, на 1 документ 2 чанка - возьмем наиближайшего соседа
df_tags_annot_result_agg = df_tags_annot_result.groupby('doc_id')['cos_scores_ka'].min().reset_index()
# Номрализуем косинусную близость - чам больше - тем ближе, т.е. лучше
df_tags_annot_result_agg['cos_scores_ka_m'] = z_logistic(df_tags_annot_result_agg['cos_scores_ka'])
# Взвешиваем скор
df_tags_annot_result_agg['cos_scores_ka_m'] = df_tags_annot_result_agg['cos_scores_ka_m'] * 0.15

# Документ, косинусная близость, нормированная косинусная близость
df_tags_annot_result_agg

,doc_id,cos_scores_ka,cos_scores_ka_m
0,doc_004,-1.107896,0.188331
1,doc_152,-1.261590,0.103932
2,doc_184,-1.263642,0.102130
3,doc_221,-1.250907,0.113248
4,doc_258,-1.299942,0.071065
5,doc_269,-1.262603,0.103043
6,doc_272,-1.317616,0.057544
7,doc_279,-1.303445,0.068269
8,doc_290,-1.297081,0.073386
9,doc_326,-1.295923,0.074335


__Prepare rag context__

In [177]:
# # Холдер для аннотации, если будет пример, где нет чанка (по бд текст), но есть документ - тогда берем аннотацию
# str_annot_to_rag = ''

# # Соберем единый датасет
# df_for_sort = df_text_result.merge(
#     df_tags_annot_result_agg, 
#     on='doc_id', 
#     how='outer'
#     )

# # Заполним пропуски в скорах
# df_for_sort['cos_scores_m'] = df_for_sort['cos_scores_m'].fillna(0)
# df_for_sort['cos_scores_ka_m'] = df_for_sort['cos_scores_ka_m'].fillna(0)

# # Единый скор и сортировка, оставляем топ SK чанков
# df_for_sort['result_score'] = df_for_sort['cos_scores_m'] + df_for_sort['cos_scores_ka_m']
# df_for_sort = df_for_sort.sort_values('result_score', ascending=False)
# target_text_chunk = df_for_sort.loc[:SK, ['top_k_indices_text', 'doc_id']]

# # Првоеряем попала ли аннотация в текст, если да - добавим ее позже
# if (target_text_chunk['top_k_indices_text'].isna()).any(): # Соберем аннотации в str_annot_to_rag - если есть пропуск
#     annot_to_rag = target_text_chunk[target_text_chunk['top_k_indices_text'].isna()]['doc_id'].unique()
#     str_annot_to_rag = '\n'.join(raw_data[raw_data['id'].isin(annot_to_rag)]['annotation'].values.tolist())

# # Собираем чанки текста из storage_t
# rag_message = [storage_t[key][1] for key in target_text_chunk['top_k_indices_text'] if key >= 0]
# rag_message = '\n'.join(rag_message)

# # Добавим анотацию, если только она попала в топ без чанка текста
# if str_annot_to_rag:
#     rag_message = rag_message + f'\n{str_annot_to_rag}'

# print(rag_message)

In [265]:
# Холдер для аннотации
annotation_doc_id = ''

# Соберем единый датасет
df_for_sort = df_text_result.merge(
    df_tags_annot_result_agg, 
    on='doc_id', 
    how='outer'
    ).merge(
    bm25_results, 
    on=['top_k_indices_text', 'doc_id'], 
    how='outer'        
    )

# Заполним пропуски в скорах
df_for_sort['cos_scores_m'] = df_for_sort['cos_scores_m'].fillna(0)
df_for_sort['cos_scores_ka_m'] = df_for_sort['cos_scores_ka_m'].fillna(0)
df_for_sort['bm25_score'] = df_for_sort['bm25_score'].fillna(0)

# Единый скор и сортировка, оставляем топ SK чанков
df_for_sort['result_score'] = df_for_sort['cos_scores_m'] + df_for_sort['cos_scores_ka_m'] + df_for_sort['bm25_score'] 
df_for_sort = df_for_sort.sort_values('result_score', ascending=False)
target_text_chunk = df_for_sort.reset_index(drop=True).loc[:SK-1, ['top_k_indices_text', 'doc_id']]

# NEW - индексы для чанков по тексту
target_text_chunk_notna = target_text_chunk[~target_text_chunk['top_k_indices_text'].isna()]

# # Собираем чанки текста из storage_t
rag_message = [storage_t[key][1] for key in target_text_chunk_notna['top_k_indices_text']]
rag_message = '\n'.join(rag_message)

# # Если есть doc_id без чанка, добавим анотацию doc_id 
if (target_text_chunk['top_k_indices_text'].isna()).any(): 
    # Получаем doc_id у пропущенного значения 
    annotation_doc_id = target_text_chunk[
        target_text_chunk['top_k_indices_text'].isna()
        ].reset_index()\
        .loc[0, 'doc_id'] 
    
    annotation_missed_chunk = data[data['id']==annotation_doc_id]['annotation'].values[0] # Аннотация пропущенного id
    rag_message += f"\nАннотация: {annotation_missed_chunk}"

# Добавим анатацию самого сонаправленного doc_id
if annotation_doc_id != target_text_chunk.loc[0, 'doc_id']:
    annotation_doc_id = target_text_chunk.loc[0, 'doc_id']
    annotation_top1 = data[data['id']==annotation_doc_id]['annotation'].values[0] # Аннотация пропущенного id
    rag_message += f"\nАннотация: {annotation_top1}"

target_text_chunk


,top_k_indices_text,doc_id
0,29.0,doc_004
1,28.0,doc_004
2,30.0,doc_004
3,31.0,doc_004
4,33.0,doc_004
5,27.0,doc_004
6,34.0,doc_004
7,26.0,doc_004
8,32.0,doc_004
9,2114.0,doc_245


In [262]:
annotation_doc_id

'doc_221'

In [266]:
print(rag_message)

В чем плюсы и минусы доверительного управления?Перед тем как обращаться к управляющему или покупать паи ПИФов, важно взвесить все за и против. **Плюсы** * Доверительное управление может принести больше дохода, чем банковский вклад. * Как и в случае с вкладом, вам не нужно постоянно следить за ситуацией на бирже, выбирать ценные бумаги или другие активы, определяться, когда их покупать, а когда продавать. Все решения за вас принимает доверительный управляющий или УК паевого фонда. **Минусы** * Прибыль не гарантирована. Инвестиции — это всегда риск, и чем больше потенциальная доходность, тем выше вероятность все потерять. Деньги в доверительном управлении не защищены государственной системой страхования вкладов. * Ваши доходы будут зависеть от решений посредника, выигрышных или неудачных. Поэтому выбирать доверительного управляющего или ПИФ стоит очень тщательно. Подробнее о рисках, с которыми сталкивается новичок на фондовом рынке, читайте в статье «Что нужно знать начинающему инвестору

In [175]:
print(rag_message)

В чем плюсы и минусы доверительного управления?Перед тем как обращаться к управляющему или покупать паи ПИФов, важно взвесить все за и против. **Плюсы** * Доверительное управление может принести больше дохода, чем банковский вклад. * Как и в случае с вкладом, вам не нужно постоянно следить за ситуацией на бирже, выбирать ценные бумаги или другие активы, определяться, когда их покупать, а когда продавать. Все решения за вас принимает доверительный управляющий или УК паевого фонда. **Минусы** * Прибыль не гарантирована. Инвестиции — это всегда риск, и чем больше потенциальная доходность, тем выше вероятность все потерять. Деньги в доверительном управлении не защищены государственной системой страхования вкладов. * Ваши доходы будут зависеть от решений посредника, выигрышных или неудачных. Поэтому выбирать доверительного управляющего или ПИФ стоит очень тщательно. Подробнее о рисках, с которыми сталкивается новичок на фондовом рынке, читайте в статье «Что нужно знать начинающему инвестору

## 4.3 Пример вызова LLM

In [68]:
rag_text_for_llm = f'\n**Для ответа используй следующие знания из RAG базы данных**:\n{rag_message}'

question = user_query + rag_text_for_llm
print('Контекст для модели:\n\n', question)

Контекст для модели:

 В чем плюсы и минусы доверительного управления?
**Для ответа используй следующие знания из RAG базы данных**:
Как выбрать доверительного управляющего?Искать нужно только среди компаний с лицензией профессионального участника рынка ценных бумаг — проверить ее наличие можно в онлайн-справочнике Банка России. В поисковых системах «Яндекс» и Mail.ru сайты финансовых посредников с лицензией Банка России имеют специальную маркировку — галочку в синем кружке. Сайты без такого знака принадлежат компаниям, которые работают нелегально. В своих офисах, на сайтах и в мобильных приложениях доверительные управляющие обязаны раскрывать информацию о себе. В частности, они должны сообщать номер лицензии и полный список финансовых услуг, а также название саморегулируемой организации, в которой состоят. Выбирая управляющего, обратите внимание на несколько параметров. **Кредитный рейтинг.** Он помогает понять, насколько компания финансово устойчива. В нем учтены динамика прибыли орг

In [69]:
# Ответ модели с RAG
responce = answer_generation(question)
print(responce)


Здравствуйте! Очень рад помочь вам разобраться в вопросе доверительного управления.

**Доверительное управление** – это передача ваших активов управляющей компании для инвестирования с целью получения прибыли. Давайте рассмотрим основные плюсы и минусы этого подхода.

**Плюсы доверительного управления:**

*   **Потенциально более высокая доходность:** Доверительное управление может принести доход больше, чем, например, банковский вклад. Это связано с тем, что инвестиции направляются в более доходные, но и более рискованные инструменты.
*   **Отсутствие необходимости самостоятельного анализа рынка:** Вам не нужно тратить время на изучение рынка, выбор ценных бумаг и принятие инвестиционных решений. Все это берет на себя профессиональный управляющий.

**Минусы доверительного управления:**

*   **Риск потери средств:** Инвестиции всегда связаны с риском, и в доверительном управлении он присутствует. Прибыль не гарантирована и есть вероятность потерять часть или даже всю сумму вложений. В